In [1]:
'''
Logistic Regression Model using TensorFlow
Used to predict categories based upon feature data

Relies on multivariate linear regression model coupled with softmax activation (logit function)
to predict categories

'''

'\nLogistic Regression Model using TensorFlow\nUsed to predict categories based upon feature data\n\nRelies on multivariate linear regression model coupled with softmax activation (logit function)\nto predict categories\n\n'

In [2]:
import tensorflow as tf

# Play with MNIST data set
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
DATA_DIR = '/tmp/data'

# Define number of training runs for the model.
NUM_STEPS = 1000
MINIBATCH_SIZE = 100

In [4]:
# Download data and save it locally; label the data using one hot encoding
data = input_data.read_data_sets(DATA_DIR, one_hot=True)
data

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


Datasets(train=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x7f8d580d64a8>, validation=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x7f8d01294f98>, test=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x7f8d01294e80>)

In [6]:
# DEFINE MODEL

# "placeholders" are input tensor parameters for a model that get initialized by our session
# x is our matrix of features (pixels)
# each image is a 28x28 pixel image.  Images represent a vector of 784 float values ranging from 0-255.
x = tf.placeholder(tf.float32, [None, 784], name='inputs')

# y is reserved for labels of data set
y_true = tf.placeholder(tf.float32, [None, 10], name='true_values')

# "Variables" are mutable objects that save state with each session run
# Need 784x10 weight values corresponding to connections between input nodes (784) and output nodes (10)
W = tf.Variable(tf.zeros([784, 10]), name='weights')

# use linear regression model; use TF matrix multiplicaiton to form our predictions
y_pred = tf.matmul(x, W, name='predicted_values')


# train model by comparing predictions with true labels
# use softmax cross entropy objective function to calculate differences
# calculate the mean error across all 10 categories 

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_pred, labels=y_true))

# use gradient descent optimizer to find optimal values to minimize error
# minimize function takes in tensor to minimize as its argument
gd_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

# define our evaluation
# tf.argmax returns index of highest value across a given axis
# tf.equal evaluates and returns a boolean
correct_mask = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y_true, 1))
accuracy = tf.reduce_mean(tf.cast(correct_mask, tf.float32))


In [12]:
with tf.Session() as sess:
    
    # Train our weights by running our computational graph model    
    
    # Create TensorBoard Output
    # Save TF graph; log cost function output;
    tf.train.write_graph(sess.graph, '.','graph.pbtxt')
    # cost_op = tf.scalar_summary("loss", cross_entropy) 
    cost_op = tf.summary.scalar("loss", cross_entropy) 
    # merged = tf.merge_all_summaries() 
    merged = tf.summary.merge_all()
    
    # Initialize variables
    sess.run(tf.initialize_all_variables())
    writer = tf.summary.FileWriter('.', sess.graph) 

    # Perform training
    for i in range(NUM_STEPS):
        batch_xs, batch_ys = data.train.next_batch(MINIBATCH_SIZE)
        sess.run(gd_step, feed_dict={x: batch_xs, y_true: batch_ys})
        
        # Collect loss metrics for TF Board
        summary_str = sess.run(cost_op, feed_dict={x: batch_xs, y_true: batch_ys})
        writer.add_summary(summary_str, i) 
    
    # Run our evaluation now that model (weights) have been trained; feed parameters: test data and labels
    ans = sess.run(accuracy, feed_dict={x: data.test.images, y_true: data.test.labels})
        

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [15]:
print("Accuracy: {:.4}%".format(ans*100))

Accuracy: 91.65%
